In [28]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import GridSearchCV, cross_val_score

# load corpus
corpus = pd.read_csv('corpus.csv.gz', compression='gzip')
corpus = corpus[corpus['qual_a_melhor_classificao_para_esse_texto:confidence'] == 1]
corpus = corpus.reset_index()

corpus_feat = pd.read_csv('corpus_liwc_mtx.csv.gz', compression='gzip')
corpus_feat.shape

(534, 68)

In [29]:
import re

def wc(x): 
    try:
        return len(re.findall(r'\w+', x['content']))
    except:
        return 0
    
corpus['wc'] = corpus.apply(wc,axis=1)
corpus_feat['wc'] = corpus['wc']

In [30]:
corpus_feat.drop('Unnamed: 0', axis=1,inplace=True)
corpus_feat.drop('confidence', axis=1,inplace=True)

In [31]:
# fix labels to binary
def classFit(x):
    if x['class'] == "diario":
        return 1
    else:
        return -1
    
corpus_feat['class_'] = corpus_feat.apply(classFit,axis=1)
target = corpus_feat['class_'].values

print(corpus_feat['class'].values[:2])
print(corpus_feat['class_'][:2])

['diario' 'outro']
0    1
1   -1
Name: class_, dtype: int64


In [32]:
wc_vector = corpus_feat['wc']
class_vector = corpus_feat['class']

corpus_feat.drop('class',axis=1,inplace=True)
corpus_feat.drop('class_',axis=1,inplace=True)
corpus_feat.drop('wc',axis=1,inplace=True)

In [33]:
data = corpus_feat.as_matrix().astype(float) / wc_vector.as_matrix().astype(float)[:, np.newaxis]
data[np.isnan(data)] = 0
data[data >= 1E308] = 0
data.shape

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.


(534, 64)

## Evaluating SVM

In [22]:
parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
grid_search = GridSearchCV(SVC(), parameters, cv=3, n_jobs=3, verbose=3, scoring='accuracy')

grid_search.fit(data, target)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] C=1, kernel=linear ..............................................
[CV] C=1, kernel=linear ..............................................
[CV] C=1, kernel=linear ..............................................
[CV] ............... C=1, kernel=linear, score=0.636872, total=   0.0s
[CV] ............... C=1, kernel=linear, score=0.646067, total=   0.0s
[CV] C=1, kernel=rbf .................................................
[CV] ............... C=1, kernel=linear, score=0.649718, total=   0.0s
[CV] C=1, kernel=rbf .................................................
[CV] C=1, kernel=rbf .................................................
[CV] .................. C=1, kernel=rbf, score=0.648045, total=   0.0s
[CV] C=10, kernel=linear .............................................
[CV] .................. C=1, kernel=rbf, score=0.646067, total=   0.0s
[CV] .................. C=1, kernel=rbf, score=0.649718, total=   0.1s
[CV] ............

[Parallel(n_jobs=3)]: Done  12 out of  12 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=3)]: Done  12 out of  12 | elapsed:    0.4s finished


GridSearchCV(cv=3, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=3,
       param_grid={'kernel': ('linear', 'rbf'), 'C': [1, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='accuracy', verbose=3)

In [23]:
print(grid_search.best_estimator_)
print(grid_search.best_score_)
print(grid_search.best_params_)

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
0.75468164794
{'C': 10, 'kernel': 'linear'}


## Evaluating Naive Bayses

In [24]:
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()

acc = cross_val_score(model, data, target, cv=10, scoring='accuracy').mean()
precision = cross_val_score(model, data, target, cv=10, scoring='precision').mean()
recall = cross_val_score(model, data, target, cv=10, scoring='recall').mean()
print(precision)
print(acc)
print(recall)

0.792230333256
0.691121055744
0.716302521008


In [25]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()

precision = cross_val_score(model, data, target, cv=10, scoring='precision').mean()
acc = cross_val_score(model, data, target, cv=10, scoring='accuracy').mean()
recall = cross_val_score(model, data, target, cv=10, scoring='recall').mean()
print(precision)
print(acc)
print(recall)

0.647960006451
0.647960006451
1.0


In [34]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score
import oll
import numpy as np

## manual 10-fold cross-validation
kf = KFold(n_splits=2, random_state=None, shuffle=False)

methods = ["P" ,"AP" ,"PA" ,"PA1","PA2" ,"PAK","CW" ,"AL"]

for m in methods:
    accuracy = []
    precision = []
    recall = []
    
    for train_index, test_index in kf.split(data):
        model = oll.oll(m, C=1)
            
        X_train, X_test = data[train_index], data[test_index]
        y_train, y_test = target[train_index], target[test_index]
        model.fit(X_train, y_train)
        predicted = model.predict(X_test)

        accuracy.append(accuracy_score(y_test, predicted))
        precision.append(precision_score(y_test, predicted))
        recall.append(recall_score(y_test, predicted))

    print(m + ': acc(' + str(np.mean(accuracy)) 
          + '), prec(' + str(np.mean(precision))
          + '), rec(' + str(np.mean(recall)) + ')'
         )

P: acc(0.644194756554), prec(0.646993064753), rec(0.991017964072)
AP: acc(0.681647940075), prec(0.707075030359), rec(0.912805673569)
PA: acc(0.64606741573), prec(0.647236067697), rec(0.997005988024)
PA1: acc(0.64606741573), prec(0.647236067697), rec(0.997005988024)
PA2: acc(0.64606741573), prec(0.647236067697), rec(0.997005988024)
PAK: acc(0.644194756554), prec(0.646526747226), rec(0.994011976048)
CW: acc(0.666666666667), prec(0.66296826533), rec(0.991017964072)
AL: acc(0.644194756554), prec(0.646526747226), rec(0.994011976048)
